<a href="https://colab.research.google.com/github/SimoneBrancato/Magistrale-Informatica/blob/main/Simone/0103%20Big%20Data/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esercizio (consegna il 7 aprile 2025)
Scaricare i dati disponibili a questo url: amazon reviews o questo amazon review dropbox
Il dataset è composto da due file: train and test in csv. Ogni file contiene le seguenti informazioni

polarity - 1 for negative and 2 for positive
title - review heading
text - review body

Generare i vettori sparsi applicando il q-shingle ai dati di training con q=3.
Sui vettori sparsi Applicare il MinHashing LSH sul dataset di training.

Usare il file di testing e applicare una k-nearest neighbor con i dati di testing su cui è stato applicato l'hashing. Usare k=3 e classificare l'elemento con del test set con la polarità maggiormente presente.

*Identificare i cluster di recensioni. Ogni cluster di recensione contiene le coppie di recensioni che hanno una similarità > di 0.6. Da svolgere dopo l'introduzione alle network

In [1]:
import kagglehub

path = kagglehub.dataset_download("kritanjalijain/amazon-reviews")

print("Path to dataset files:", path)

100%|██████████| 1.29G/1.29G [00:44<00:00, 31.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2


In [2]:
import pandas as pd
import os

print("File: ", os.listdir(path))

File:  ['amazon_review_polarity_csv.tgz', 'test.csv', 'train.csv']


In [3]:
q = 3

def shingle(text: str, k: int):
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

In [4]:
train_df = pd.read_csv("/root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2/train.csv", header=None, names=["polarity", "title", "text"])
test_df = pd.read_csv("/root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2/test.csv", header=None, names=["polarity", "title", "text"])

In [16]:
shingles_list= []

for _,row in train_df.iloc[:100].iterrows():
  full_text = f"{row['title']} {row['text']}"
  record_shingles = shingle(full_text, q)
  shingles_list.append(record_shingles)


In [18]:
for shingles in shingles_list[:5]:  # Mostriamo solo i primi 5
    print("Shingles:", shingles)
    print("-----")

Shingles: {'end', 'e v', ' vi', 'ron', ' ba', 'tar', 'tun', 'e n', ' mi', ' tr', 'ul ', 'ul!', 'esh', '. I', 'din', 'she', 'one', ' im', 'nin', 'o l', 'r T', ' ha', 'key', 'le ', '-ga', 'It ', 'rat', 'rec', 'fre', 'ave', 's b', 'l! ', 'n! ', 'to ', 'men', 'no ', 'ing', 'ain', 'th ', ' of', 'uld', 'eco', 'he ', 'ver', 'm c', ' so', ' fo', 'p w', 'o c', 'boa', 'on-', 'gam', 'o p', 'min', ' mu', 'e k', 'ck ', 'ros', 'ic!', 'out', 'sou', 'I w', 'vid', 'es ', 'wit', 'ur ', '. g', 'e p', 't w', 'sic', 'ras', 'id.', 't h', 'uti', 'ld ', ' bu', 'o C', 'has', 'rs ', 'cks', 'e g', 'rom', ' re', 'ate', 'of ', 'tak', 'tif', 'lfu', ' al', 'pre', 'sen', 'f t', 'rch', 'ack', 'ell', 'opl', 'ifu', 'r p', ' pe', 'e b', 'aye', ' gr', 'str', 'ene', 'ss ', 'ind', 'e s', 'yed', ' wh', ' Th', 'om ', 'rud', 'lis', 's I', 'Thi', 'ts ', 'll ', ' fr', 'n y', 'but', 'orc', 'and', 'bac', 'mes', 'wel', 'ess', 'Cro', 'ita', 'oss', 'ho ', 'o w', 'und', 'n f', 'usi', ' or', 't e', 's a', 'nd ', 'hes', 'for', 'hat', 'd

In [21]:
vocab = set()

for shingles in shingles_list:
  vocab.update(shingles)
print(vocab)

{'yss', 'MH ', 'et.', 'd v', 'ed]', 'dub', 'an"', ' ba', 'ulw', 'lpf', 'e n', '5 S', 'die', 'Gal', ' iP', 'ar!', 'orm', 'd k', 'lub', 'ryb', 'o b', 'e A', 'w 1', 'o l', ' Fu', 'le.', 'jea', ' ex', 'zle', 'A".', 'g y', 'llm', ' ap', 'uzz', 'mph', 'rat', 'fer', 'Bat', 'l H', 'r) ', 'sh-', ' Ai', 'Gra', 'ura', '. U', '/MT', 'Wel', 'You', ' **', 'fic', 'ang', 'ew ', 'pta', 'rt-', 'ipl', 'lso', 'oo.', 'ght', 'y m', 'atl', 'm s', 'ow ', 'e -', ', J', 'eco', 'ici', 'D o', 'Two', 'V??', 'udg', 'wne', 'i h', 'r, ', 'ylo', 'maz', ' ! ', 'sap', 'd I', 'two', 'ps3', 'I a', '/ba', 'or)', 'ign', ' da', 'A D', 'vey', 'lab', 'u p', 'o a', 'g J', ' On', '"ca', 'k N', 'efo', 'une', 'bov', 'lad', 'y e', 'Phe', 'w -', 'u (', 'eng', 'ABC', 'oo ', 'on:', 'hli', 'TIV', 'd l', 'rit', 'nsh', 'G s', 'aby', 'dan', ' lu', 't w', 'Wai', '"or', ' MI', 'l v', 'elb', 'efi', 'ool', 'ath', 'ous', 'Ed ', 'un,', 'u b', 'o N', ' "A', 'wea', 'ld ', 'bje', 'chi', 'rk!', 'mmi', 'my ', 'gre', 's :', 'rs ', 'ck,', 's .', ': I'

In [22]:
# Creazione dei vettori sparsi per ogni chiave
sparse_vectors = []

for shingles in shingles_list:
  sparse_vectors.append([1 if x in shingles else 0 for x in vocab])

# Stampa un esempio
for vec in sparse_vectors[:5]:
    print(vec)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [15]:
# Adesso applichiamo il MinHash

from random import shuffle

hash_ex = list(range(1, len(vocab)+1))
shuffle(hash_ex)  # Randomizziamo il range
print(hash_ex)

[4990, 381, 1048, 4484, 3551, 2915, 3569, 4604, 232, 4332, 833, 749, 1170, 3035, 5030, 3337, 1555, 4848, 4787, 3265, 3389, 5014, 4908, 523, 27, 5176, 2905, 4744, 2377, 4610, 2805, 1725, 1778, 4167, 1399, 2783, 4441, 2602, 2473, 582, 3436, 4191, 1129, 4318, 4202, 1941, 4080, 2970, 87, 2576, 744, 1510, 9, 4262, 4378, 4546, 3854, 4409, 859, 2470, 3456, 1754, 2913, 1876, 236, 89, 3601, 2279, 2995, 3967, 2821, 4151, 4792, 3194, 1874, 966, 1462, 3241, 705, 4028, 3669, 2896, 3743, 4773, 748, 2598, 4916, 4059, 174, 1014, 3728, 4111, 3724, 532, 2679, 1856, 2044, 4975, 557, 3481, 4338, 1105, 4936, 1002, 625, 4294, 328, 4905, 198, 792, 3971, 216, 1682, 1022, 2125, 2212, 2448, 4117, 1678, 4888, 3558, 61, 3080, 4778, 3344, 1155, 3459, 3391, 3841, 355, 3506, 4715, 4310, 3505, 4673, 4752, 5323, 896, 2613, 3388, 1215, 708, 1661, 4972, 3026, 4940, 680, 15, 2889, 1670, 3745, 5235, 2485, 453, 618, 2425, 2553, 592, 4320, 3917, 3583, 1735, 735, 2407, 555, 2500, 282, 4570, 4323, 408, 3320, 4770, 3886, 363, 